# TASK 1

In [2]:
class HashTableChaining:
    def __init__(self, size):
        self.size = size
        self.table = [[] for _ in range(size)]

    def _hash(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        index = self._hash(key)
        for pair in self.table[index]:
            if pair[0] == key:
                pair[1] = value
                return
        self.table[index].append([key, value])

    def get(self, key):
        index = self._hash(key)
        for pair in self.table[index]:
            if pair[0] == key:
                return pair[1]
        return None

    def delete(self, key):
        index = self._hash(key)
        for i, pair in enumerate(self.table[index]):
            if pair[0] == key:
                del self.table[index][i]
                return

ht = HashTableChaining(10)
ht.insert("name", "Alice")
ht.insert("age", 25)
print(ht.get("name"))  
ht.delete("age")
print(ht.get("age")) 

Alice
None


In [4]:
class HashTableLinearProbing:
    def __init__(self, size):
        self.size = size
        self.table = [None] * size

    def _hash(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        index = self._hash(key)
        while self.table[index] is not None and self.table[index][0] != key:
            index = (index + 1) % self.size
        self.table[index] = (key, value)

    def get(self, key):
        index = self._hash(key)
        original_index = index
        while self.table[index] is not None:
            if self.table[index][0] == key:
                return self.table[index][1]
            index = (index + 1) % self.size
            if index == original_index:
                break
        return None

    def delete(self, key):
        index = self._hash(key)
        original_index = index
        while self.table[index] is not None:
            if self.table[index][0] == key:
                self.table[index] = None
                return
            index = (index + 1) % self.size
            if index == original_index:
                break


ht_lp = HashTableLinearProbing(10)
ht_lp.insert("name", "Alice")
ht_lp.insert("age", 25)
print(ht_lp.get("name")) 
ht_lp.delete("age")
print(ht_lp.get("age"))  

Alice
None


# TASK 2

In [7]:
import matplotlib.pyplot as plt
import random
import string

def custom_hash(key, table_size=100):
    """Custom hash function using polynomial rolling method."""
    prime = 31
    hash_value = 0
    for char in key:
        hash_value = (hash_value * prime + ord(char)) % table_size
    return hash_value


print(custom_hash("hello")) 

22


In [9]:
def measure_collisions(hash_function, num_keys=1000, table_size=100):
    hash_table = [0] * table_size
    keys = [''.join(random.choices(string.ascii_letters, k=5)) for _ in range(num_keys)]  

    for key in keys:
        index = hash_function(key, table_size)
        hash_table[index] += 1 

    return hash_table

custom_collisions = measure_collisions(custom_hash)
python_collisions = measure_collisions(lambda key, size: hash(key) % size)

print(f"Custom Hash Collisions: {sum(1 for x in custom_collisions if x > 1)}")
print(f"Python Hash Collisions: {sum(1 for x in python_collisions if x > 1)}")


Custom Hash Collisions: 100
Python Hash Collisions: 100


# TASK 3

In [12]:
from collections import OrderedDict

class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = OrderedDict()

    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        self.cache.move_to_end(key)  
        return self.cache[key]

    def put(self, key: int, value: int):
        if key in self.cache:
            self.cache.move_to_end(key) 
        elif len(self.cache) >= self.capacity:
            self.cache.popitem(last=False) 
        self.cache[key] = value

cache = LRUCache(2)
cache.put(1, "A")
cache.put(2, "B")
print(cache.get(1)) 
cache.put(3, "C")  
print(cache.get(2))  

A
-1


In [14]:
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

class LRUCacheLinkedList:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}  
        self.head, self.tail = Node(0, 0), Node(0, 0)  
        self.head.next = self.tail
        self.tail.prev = self.head

    def _remove(self, node):
        """Remove node from linked list"""
        node.prev.next, node.next.prev = node.next, node.prev

    def _insert(self, node):
        """Insert node at the end (most recently used)"""
        node.prev, node.next = self.tail.prev, self.tail
        self.tail.prev.next = node
        self.tail.prev = node

    def get(self, key: int) -> int:
        if key in self.cache:
            node = self.cache[key]
            self._remove(node) 
            self._insert(node)
            return node.value
        return -1

    def put(self, key: int, value: int):
        if key in self.cache:
            self._remove(self.cache[key])  
        elif len(self.cache) >= self.capacity:
            lru_node = self.head.next  
            self._remove(lru_node)
            del self.cache[lru_node.key]

        new_node = Node(key, value)
        self.cache[key] = new_node
        self._insert(new_node)


cache_ll = LRUCacheLinkedList(2)
cache_ll.put(1, "A")
cache_ll.put(2, "B")
print(cache_ll.get(1)) 
cache_ll.put(3, "C") 
print(cache_ll.get(2))  

A
-1


In [16]:
import time

def test_cache_performance(cache_class, capacity, operations):
    cache = cache_class(capacity)
    start_time = time.time()
    for i in range(operations):
        cache.put(i, i) 
        cache.get(i)     
    return time.time() - start_time

capacity = 1000
operations = 10000

ordered_dict_time = test_cache_performance(LRUCache, capacity, operations)
linked_list_time = test_cache_performance(LRUCacheLinkedList, capacity, operations)

print(f"OrderedDict LRU Time: {ordered_dict_time:.6f} sec")
print(f"Linked List LRU Time: {linked_list_time:.6f} sec")

OrderedDict LRU Time: 0.008604 sec
Linked List LRU Time: 0.016683 sec
